# Hyperparameter Optimization

In this tutorial, we first demonstrate how `P3alphaRecommender`'s performance can be optimized
by [optuna](https://github.com/optuna/optuna)-backed `P3alphaOptimizer`.

Then, by further splitting the ground-truth interaction into tran, validation and test ones,
we compare several recommenders' performance optimized on the validation set and measured on the test set.

In [1]:
import numpy as np
import scipy.sparse as sps
from sklearn.model_selection import train_test_split

from irspack.dataset.movielens import MovieLens1MDataManager
from irspack.recommenders import P3alphaRecommender
from irspack.optimizers import P3alphaOptimizer
from irspack.split import rowwise_train_test_split
from irspack.evaluator import Evaluator

## Read the ML1M dataset again.

We again prepare the sparse matrix `X`.

In [2]:
loader = MovieLens1MDataManager()

df = loader.read_interaction()

movies = loader.read_item_info()
movies.head()

unique_user_ids, user_index = np.unique(df.userId, return_inverse=True)
unique_movie_ids, movie_index = np.unique(df.movieId, return_inverse=True)

movie_id_vs_movie_index = { mid: i for i, mid in enumerate(unique_movie_ids)}

X = sps.csr_matrix(
    (
        np.ones(df.shape[0]),
        ( user_index, movie_index)
    )
)

## Split scheme 2. Hold-out for partial users.

To perform the hyperparameter optimization, we have to repeatedly measure the accuracy metrics on the validation set. As mentioned in the previous tutorial, doing this for all users is time-comsuming (often heavier than the recommender's learning process), so we truncate this subset as follows:

1. First split **users** into "train", "validation" (and "test") ones.
1. For train users, feed all their interactions into the recommender. For validation (test) users, hold-out part of their interaction for the validation ("prediction" part), and feed the rest ("learning" part) into the recommender.
1. After the fit, ask the recommender to output the score only for validation (test) users, and see how it ranks these held-out interactions for the validation (test) users.

![Perform hold out for part of users.](./split2.png "split1")

Although we have prepared another function to do this procedure, let us first do this manually.

In [3]:
# Split users into train and validation users.

X_train_user, X_valid_user = train_test_split(X, test_size=.4, random_state=0)

# Split the validation users' interaction into learning 50% and predcition 50%.

X_valid_learn, X_valid_predict = rowwise_train_test_split(
    X_valid_user, test_ratio=.5, random_state=0
)

## Define the evaluator and optimize the validation metric

As illustrated above, we will use 

 * Train users' all interactions (``X_train_user``)
 * Validation users' 50% interaction (``X_valid_learn``)
 
as the recommender's training resource, and validation users' rest interaction (``X_valid_predict``) as the held-out ground truth:

In [4]:
X_train_val_learn = sps.vstack([X_train_user, X_valid_learn])
evaluator = Evaluator(X_valid_predict, offset=X_train_user.shape[0], target_metric='ndcg', cutoff=20)

The ``offset`` parameter specifies where the validation user block begins (where the train user block ends).

Now to start the optimization.

In [5]:
if True:
    # Truncating the stderr output as it's a bit lengthy to show in the documentation.
    # When you run this note book, you don't have to truncate it.
    from irspack.utils.default_logger import disable_default_handler
    import optuna.logging
    disable_default_handler()
    optuna.logging.disable_default_handler() 
    
optimizer = P3alphaOptimizer(X_train_val_learn, evaluator)
best_params, validation_results = optimizer.optimize(random_seed=0, n_trials=20)

The best `ndcg@20` value is


In [6]:
validation_results['ndcg@20'].max()

0.515800664255095

which has been obtained by using these hyper parameters:

In [7]:
best_params

{'top_k': 279, 'normalize_weight': True}

Meanwhile, the default argument of ``P3alphaRecommdner`` (which has been used so far)
attains `ndcg@20` = 0.403. So this is indeed a significant improvement:

In [8]:
rec_default = P3alphaRecommender(X_train_val_learn).learn()
evaluator.get_score(rec_default)['ndcg']

0.408406019199828

## Check the recommender's output again

Let us check how our recommender has evolved from the first tutorial. We consider the same setting (a new user has watched "Toy Story"), but fit the 
recommender using the obtained parameters.

In [9]:
rec_tuned = P3alphaRecommender(X, **best_params).learn()

from irspack.utils.id_mapping import IDMappedRecommender
id_mapped_rec = IDMappedRecommender(
    rec_tuned, user_ids=unique_user_ids, item_ids=unique_movie_ids
)

In [10]:
toystory_id = 1
recommended_id_and_score = id_mapped_rec.get_recommendation_for_new_user(
    user_profile=[toystory_id], cutoff=10
)

# Top-10 recommendations
movies.reindex([movie_id for movie_id, score in recommended_id_and_score])

,title,genres,release_year
movieId,,,
1265,Groundhog Day (1993),Comedy|Romance,1993
2396,Shakespeare in Love (1998),Comedy|Romance,1998
3114,Toy Story 2 (1999),Animation|Children's|Comedy,1999
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi,1977
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War,1983
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,1980
1270,Back to the Future (1985),Comedy|Sci-Fi,1985
2028,Saving Private Ryan (1998),Action|Drama|War,1998
34,Babe (1995),Children's|Comedy|Drama,1995


Note how drastically the recommended contents have changed (increased significance of genre "Children's" and disapperance of "Star Wars" series, etc...).

## A train/validation/test split example

To rigorously compare the performance of various recommender algorithms,
we should measure the final score against the **test** dataset, not the validation set,
and it is straightforward now.

To begin with, we have prepared a function called ``split_dataframe_partial_user_holdout`` which
splits the users in the original dataframe into train/validation/test users,
holding out partial interaction for validation/test user:

In [11]:
from irspack.split import split_dataframe_partial_user_holdout

dataset, item_ids = split_dataframe_partial_user_holdout(
    df, 'userId', 'movieId', val_user_ratio=.3, test_user_ratio=.3,
    heldout_ratio_val=.5, heldout_ratio_test=.5
)

dataset

{'train': <irspack.split.userwise.UserTrainTestInteractionPair at 0x7fdd1223a9a0>,
 'val': <irspack.split.userwise.UserTrainTestInteractionPair at 0x7fdd120a1a30>,
 'test': <irspack.split.userwise.UserTrainTestInteractionPair at 0x7fdd120b3dc0>}

As you can see, the returned ``dataset`` is a dictionary which stores train/validation/test-users' interactions as an instance of ``UserTrainTestInteractionPair``.

In [12]:
train_users = dataset['train']
val_users = dataset['val']
test_users = dataset['test']

# Concatenate train/validation users into one.
train_and_val_users = train_users.concat(val_users)

In [13]:
val_users.X_train

<1812x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 150090 stored elements in Compressed Sparse Row format>

In [14]:
val_users.X_test

<1812x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 149227 stored elements in Compressed Sparse Row format>

In [15]:
val_users.X_all # which equals val_users.X_train + val_users.X_test

<1812x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 299317 stored elements in Compressed Sparse Row format>

In [16]:
# For train users, there is no "test" interaction held out.
train_users.X_test

<2416x3706 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

For each recommender algorithm (here ``P3alpha``, ``RP3beta``, ``IALS`` and ``DenseSLIM``), we perform:

1. Hyperparameter optimization. During this phase, we will be using train users' all interaction and validation
   users' train interaction as the source of learning, and validation users' test interaction as the held-out ground truth.
2. Evaluation. During this phase, we will include train/validation users' all interactions
   as well as test users' train interaction as the source of learning,
   and fit the model using the parameters obtained in the optimization phase.
   Then we measure the recommender's performance against **test** users' test interaction.

In [17]:
from irspack.optimizers import DenseSLIMOptimizer, RP3betaOptimizer, IALSOptimizer

In [18]:
val_evaluator = Evaluator(
    val_users.X_test,
    offset=train_users.n_users,
    cutoff=20, target_metric="ndcg"
)
test_evaluator = Evaluator(
    test_users.X_test,
    offset=train_and_val_users.n_users
)
test_results = []
for optimizer_class in [IALSOptimizer, RP3betaOptimizer, P3alphaOptimizer, DenseSLIMOptimizer]:
    print(f'Start running {optimizer_class.__name__}.')
    optimizer_ = optimizer_class(
        sps.vstack([train_users.X_all, val_users.X_train]),
        val_evaluator
    )
    best_params, validation_results_df = optimizer_.optimize(n_trials=40, random_seed=0)
    recommender = optimizer_class.recommender_class(
        sps.vstack([train_and_val_users.X_all, test_users.X_train]),
        **best_params
    ).learn()

    test_score = dict(
        algorithm=optimizer_class.__name__, 
        **test_evaluator.get_scores(recommender, cutoffs=[20])
    )
    test_results.append(test_score)

Start running IALSOptimizer.


Start running RP3betaOptimizer.
Start running P3alphaOptimizer.
Start running DenseSLIMOptimizer.


As you can see, iALS outperforms others
in terms of both accuracy measures (recall, ndcg, map) and diversity measures (entropy, gini_index, appeared_item).

In [19]:
import pandas as pd
pd.DataFrame(test_results)

,algorithm,hit@20,recall@20,ndcg@20,map@20,precision@20,gini_index@20,entropy@20,appeared_item@20
0,IALSOptimizer,0.996689,0.215256,0.568571,0.140380,0.516832,0.910734,6.046934,1150.0
1,RP3betaOptimizer,0.995033,0.197060,0.532074,0.124262,0.480436,0.946186,5.484655,1122.0
2,P3alphaOptimizer,0.992274,0.188999,0.517922,0.118043,0.467964,0.962089,5.184107,730.0
3,DenseSLIMOptimizer,0.995033,0.210815,0.563781,0.137498,0.511645,0.931602,5.784477,993.0
